In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) # Display all columns in a pandas dataframe

cwd = Path(os.getcwd())
print(os.listdir("/"))

['bin', 'boot', 'dev', 'etc', 'home', 'lib', 'lib64', 'media', 'mnt', 'opt', 'proc', 'root', 'run', 'sbin', 'srv', 'sys', 'tmp', 'usr', 'var', 'get-docker.sh', 'dxdata-0.36.1-py2.py3-none-any.whl', 'install_r_kernel.R', 'install_r_packages.R', '.dockerenv']


In [33]:
# Read data
data = pd.read_csv("/mnt/project/data/densitometry_participant.csv")
data.rename(columns={
    "Participant ID": "patientId", 
    "Date of attending assessment centre | Instance 2": "date_i2", 
    "Date of attending assessment centre | Instance 3": "date_i3",
    "Age when attended assessment centre | Instance 2": "age_i2"
}, inplace=True)
data = data.sort_values(by=["patientId", "date_i2"])
data

,patientId,date_i2,date_i3,Height | Instance 2,Weight | Instance 2,Body mass index (BMI) | Instance 2,Had menopause | Instance 2,Had menopause | Instance 1,Had menopause | Instance 0,age_i2,L1-L4 BMD (bone mineral density) T-score | Instance 2,Femur neck BMD (bone mineral density) T-score (left) | Instance 2,Femur total BMD (bone mineral density) T-score (left) | Instance 2,L1-L4 TBS (trabecular bone score) | Instance 2,Femur neck BMD (bone mineral density) T-score (left) | Instance 3,Femur neck BMD (bone mineral density) T-score (right) | Instance 2,Femur neck BMD (bone mineral density) T-score (right) | Instance 3,Femur total BMD (bone mineral density) T-score (left) | Instance 3,Femur total BMD (bone mineral density) T-score (right) | Instance 2,Femur total BMD (bone mineral density) T-score (right) | Instance 3,L1-L4 BMD (bone mineral density) T-score | Instance 3
27426,1000031,2021-10-21,NaN,153.0,73.5,31.8127,Yes,NaN,Yes,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15545,1000344,2015-02-23,NaN,154.0,48.3,20.6331,Yes,Not sure - other reason,No,58,-0.719586,-0.551470,-0.713834,1.200,NaN,-0.863189,NaN,NaN,-0.928921,NaN,NaN
3403,1000621,2014-10-30,NaN,159.0,56.6,23.1094,Not sure - had a hysterectomy,Not sure - had a hysterectomy,Not sure - had a hysterectomy,58,-0.772791,-0.889445,-1.003140,1.119,NaN,-1.180800,NaN,NaN,-1.216780,NaN,NaN
23342,1000898,2018-11-07,NaN,168.0,85.9,31.1729,Yes,Yes,No,59,-0.309509,-0.759057,-0.635792,1.095,NaN,-0.845577,NaN,NaN,-0.813459,NaN,NaN
6058,1000946,2018-02-12,NaN,165.0,84.9,31.1846,Yes,NaN,Yes,67,0.548010,0.212610,0.504118,1.070,NaN,-0.246668,NaN,NaN,0.219435,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,6023313,2014-11-20,NaN,159.0,69.3,28.1147,Yes,NaN,Yes,72,-0.681709,-1.592420,-1.481710,1.099,NaN,-1.480840,NaN,NaN,-1.844660,NaN,NaN
15790,6023403,2017-09-26,NaN,162.0,75.1,28.6161,Yes,NaN,Yes,75,NaN,NaN,NaN,1.030,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10885,6023555,2019-06-20,NaN,157.0,66.2,27.5546,Yes,NaN,No,63,-0.445347,-0.754895,-0.379088,NaN,NaN,-0.502770,NaN,NaN,-0.138536,NaN,NaN
18948,6023580,2019-10-25,NaN,165.0,64.4,23.9441,Yes,NaN,No,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
def reformat_column_names(df):
    name_mapping = {}
    for col in df.columns:
        if "Instance" in col:
            col_name, instance = col.split(" | ")
            name = col_name.split(" (")[0]
            if len(col_name.split(" (")) > 2:
                name += "_" + col_name.split(" (")[-1][:-1]
            name = name.replace(" ", "_").lower()
            instance_nr = instance.split(" ")[1]
            name_mapping[col] = f"{name}_i{instance_nr}"
    return df.rename(columns=name_mapping)

data = reformat_column_names(data)
data

,patientId,date_i2,date_i3,height_i2,weight_i2,body_mass_index_i2,had_menopause_i2,had_menopause_i1,had_menopause_i0,age_i2,l1-l4_bmd_i2,femur_neck_bmd_left_i2,femur_total_bmd_left_i2,l1-l4_tbs_i2,femur_neck_bmd_left_i3,femur_neck_bmd_right_i2,femur_neck_bmd_right_i3,femur_total_bmd_left_i3,femur_total_bmd_right_i2,femur_total_bmd_right_i3,l1-l4_bmd_i3
27426,1000031,2021-10-21,NaN,153.0,73.5,31.8127,Yes,NaN,Yes,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15545,1000344,2015-02-23,NaN,154.0,48.3,20.6331,Yes,Not sure - other reason,No,58,-0.719586,-0.551470,-0.713834,1.200,NaN,-0.863189,NaN,NaN,-0.928921,NaN,NaN
3403,1000621,2014-10-30,NaN,159.0,56.6,23.1094,Not sure - had a hysterectomy,Not sure - had a hysterectomy,Not sure - had a hysterectomy,58,-0.772791,-0.889445,-1.003140,1.119,NaN,-1.180800,NaN,NaN,-1.216780,NaN,NaN
23342,1000898,2018-11-07,NaN,168.0,85.9,31.1729,Yes,Yes,No,59,-0.309509,-0.759057,-0.635792,1.095,NaN,-0.845577,NaN,NaN,-0.813459,NaN,NaN
6058,1000946,2018-02-12,NaN,165.0,84.9,31.1846,Yes,NaN,Yes,67,0.548010,0.212610,0.504118,1.070,NaN,-0.246668,NaN,NaN,0.219435,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,6023313,2014-11-20,NaN,159.0,69.3,28.1147,Yes,NaN,Yes,72,-0.681709,-1.592420,-1.481710,1.099,NaN,-1.480840,NaN,NaN,-1.844660,NaN,NaN
15790,6023403,2017-09-26,NaN,162.0,75.1,28.6161,Yes,NaN,Yes,75,NaN,NaN,NaN,1.030,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10885,6023555,2019-06-20,NaN,157.0,66.2,27.5546,Yes,NaN,No,63,-0.445347,-0.754895,-0.379088,NaN,NaN,-0.502770,NaN,NaN,-0.138536,NaN,NaN
18948,6023580,2019-10-25,NaN,165.0,64.4,23.9441,Yes,NaN,No,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
def find_menopause_status(row):
    if row["had_menopause_i0"] == "Yes" or\
    row["had_menopause_i1"] == "Yes" or\
    row["had_menopause_i2"] == "Yes":
        return True
    else:
        return False

In [36]:
# Forward fill menopause status and store value in a new column
menopausal_variables = ["had_menopause_i0", "had_menopause_i1", "had_menopause_i2"]
data["postmenopausal"] = data[menopausal_variables].apply(lambda x: find_menopause_status(x), axis=1)

# Select only postmenopausal women
data_pm = data[data["postmenopausal"]]
# Drop menopause columns
data_pm = data_pm.drop(menopausal_variables + ["postmenopausal"], axis=1).reset_index(drop=True)
data_pm

,patientId,date_i2,date_i3,height_i2,weight_i2,body_mass_index_i2,age_i2,l1-l4_bmd_i2,femur_neck_bmd_left_i2,femur_total_bmd_left_i2,l1-l4_tbs_i2,femur_neck_bmd_left_i3,femur_neck_bmd_right_i2,femur_neck_bmd_right_i3,femur_total_bmd_left_i3,femur_total_bmd_right_i2,femur_total_bmd_right_i3,l1-l4_bmd_i3
0,1000031,2021-10-21,NaN,153.0,73.5,31.8127,79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000344,2015-02-23,NaN,154.0,48.3,20.6331,58,-0.719586,-0.551470,-0.713834,1.200,NaN,-0.863189,NaN,NaN,-0.928921,NaN,NaN
2,1000898,2018-11-07,NaN,168.0,85.9,31.1729,59,-0.309509,-0.759057,-0.635792,1.095,NaN,-0.845577,NaN,NaN,-0.813459,NaN,NaN
3,1000946,2018-02-12,NaN,165.0,84.9,31.1846,67,0.548010,0.212610,0.504118,1.070,NaN,-0.246668,NaN,NaN,0.219435,NaN,NaN
4,1001273,2019-09-14,NaN,164.0,91.0,33.4252,54,1.435990,1.948020,1.612680,NaN,NaN,2.594330,NaN,NaN,1.683660,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23532,6023313,2014-11-20,NaN,159.0,69.3,28.1147,72,-0.681709,-1.592420,-1.481710,1.099,NaN,-1.480840,NaN,NaN,-1.844660,NaN,NaN
23533,6023403,2017-09-26,NaN,162.0,75.1,28.6161,75,NaN,NaN,NaN,1.030,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23534,6023555,2019-06-20,NaN,157.0,66.2,27.5546,63,-0.445347,-0.754895,-0.379088,NaN,NaN,-0.502770,NaN,NaN,-0.138536,NaN,NaN
23535,6023580,2019-10-25,NaN,165.0,64.4,23.9441,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
data_pm.isna().sum()

patientId                       0
date_i2                         0
date_i3                     21366
height_i2                       5
weight_i2                     740
body_mass_index_i2            863
age_i2                          0
l1-l4_bmd_i2                 6981
femur_neck_bmd_left_i2       6118
femur_total_bmd_left_i2      6167
l1-l4_tbs_i2                10928
femur_neck_bmd_left_i3      22759
femur_neck_bmd_right_i2      6763
femur_neck_bmd_right_i3     22785
femur_total_bmd_left_i3     22759
femur_total_bmd_right_i2     6821
femur_total_bmd_right_i3    22788
l1-l4_bmd_i3                22800
dtype: int64

In [38]:
def find_tscore(df):
    for i, row in df.iterrows():
        # Find tscore at lumbar spine
        tscore_ls = row["l1-l4_bmd_i2"]
        if np.isnan(tscore_ls):
            tscore_ls = row["l1-l4_bmd_i3"]
        
        # Find tscore at femoral neck
        tscore_neck = row[["femur_neck_bmd_left_i2", "femur_neck_bmd_right_i2"]].min()
        if np.isnan(tscore_neck):
            tscore_neck = row[["femur_neck_bmd_left_i3", "femur_neck_bmd_right_i3"]].min()
        
        # Find tscore at total hip
        tscore_totalHip = row[["femur_total_bmd_left_i2", "femur_total_bmd_right_i2"]].min()
        if np.isnan(tscore_totalHip):
            tscore_totalHip = row[["femur_total_bmd_left_i3", "femur_total_bmd_right_i3"]].min()
            
        # Save tscores in dataframe
        df.loc[i, "tscore_ls"] = tscore_ls
        df.loc[i, "tscore_neck"] = tscore_neck
        df.loc[i, "tscore_totalHip"] = tscore_totalHip
    
    # Drop old columns and only keep new ones
    df.drop([
        "l1-l4_bmd_i2", "l1-l4_bmd_i3",
        "femur_neck_bmd_left_i2", "femur_neck_bmd_right_i2", "femur_neck_bmd_left_i3", "femur_neck_bmd_right_i3",
        "femur_total_bmd_left_i2", "femur_total_bmd_right_i2", "femur_total_bmd_left_i3", "femur_total_bmd_right_i3",
    ], axis=1, inplace=True)
    return df

In [39]:
data_pm = find_tscore(data_pm)
data_pm

,patientId,date_i2,date_i3,height_i2,weight_i2,body_mass_index_i2,age_i2,l1-l4_tbs_i2,femur_total_bmd_right_i3,tscore_ls,tscore_neck,tscore_totalHip
0,1000031,2021-10-21,NaN,153.0,73.5,31.8127,79,NaN,NaN,NaN,NaN,NaN
1,1000344,2015-02-23,NaN,154.0,48.3,20.6331,58,1.200,NaN,-0.719586,-0.863189,-0.928921
2,1000898,2018-11-07,NaN,168.0,85.9,31.1729,59,1.095,NaN,-0.309509,-0.845577,-0.813459
3,1000946,2018-02-12,NaN,165.0,84.9,31.1846,67,1.070,NaN,0.548010,-0.246668,0.219435
4,1001273,2019-09-14,NaN,164.0,91.0,33.4252,54,NaN,NaN,1.435990,1.948020,1.612680
...,...,...,...,...,...,...,...,...,...,...,...,...
23532,6023313,2014-11-20,NaN,159.0,69.3,28.1147,72,1.099,NaN,-0.681709,-1.592420,-1.844660
23533,6023403,2017-09-26,NaN,162.0,75.1,28.6161,75,1.030,NaN,NaN,NaN,NaN
23534,6023555,2019-06-20,NaN,157.0,66.2,27.5546,63,NaN,NaN,-0.445347,-0.754895,-0.379088
23535,6023580,2019-10-25,NaN,165.0,64.4,23.9441,57,NaN,NaN,NaN,NaN,NaN


In [42]:
data_pm.drop("femur_total_bmd_right_i3", axis=1, inplace=True)  # no longer needed
data_pm.isna().sum()

patientId                 0
date_i2                   0
height_i2                 5
weight_i2               740
body_mass_index_i2      863
age_i2                    0
l1-l4_tbs_i2          10928
tscore_ls              6898
tscore_neck            6015
tscore_totalHip        6028
dtype: int64

In [45]:
data_pm = data_pm.rename(columns={"l1-l4_tbs_i2": "tbs_ls"})
data_pm_no_tscore_nans = data_pm.dropna(subset=["tscore_ls", "tscore_neck", "tscore_totalHip"])
data_pm_no_tscore_nans

,patientId,date_i2,height_i2,weight_i2,body_mass_index_i2,age_i2,tbs_ls,tscore_ls,tscore_neck,tscore_totalHip
1,1000344,2015-02-23,154.0,48.3,20.6331,58,1.200,-0.719586,-0.863189,-0.928921
2,1000898,2018-11-07,168.0,85.9,31.1729,59,1.095,-0.309509,-0.845577,-0.813459
3,1000946,2018-02-12,165.0,84.9,31.1846,67,1.070,0.548010,-0.246668,0.219435
4,1001273,2019-09-14,164.0,91.0,33.4252,54,NaN,1.435990,1.948020,1.612680
5,1001554,2018-03-22,153.0,56.7,24.2215,67,1.017,-2.730180,-0.617614,-0.995070
...,...,...,...,...,...,...,...,...,...,...
23528,6022316,2017-09-19,170.0,77.5,27.4589,73,1.237,0.424648,-0.708455,-0.463218
23530,6023012,2019-06-26,169.0,76.1,26.7713,64,NaN,-1.638690,-1.565910,-1.563450
23531,6023119,2015-10-02,161.0,62.7,23.8912,57,1.294,1.472440,0.914185,1.294470
23532,6023313,2014-11-20,159.0,69.3,28.1147,72,1.099,-0.681709,-1.592420,-1.844660


In [46]:
data_pm_no_nans = data_pm.dropna(subset=["tscore_ls", "tscore_neck", "tscore_totalHip", "tbs_ls"])
data_pm_no_nans

,patientId,date_i2,height_i2,weight_i2,body_mass_index_i2,age_i2,tbs_ls,tscore_ls,tscore_neck,tscore_totalHip
1,1000344,2015-02-23,154.0,48.3,20.6331,58,1.200,-0.719586,-0.863189,-0.928921
2,1000898,2018-11-07,168.0,85.9,31.1729,59,1.095,-0.309509,-0.845577,-0.813459
3,1000946,2018-02-12,165.0,84.9,31.1846,67,1.070,0.548010,-0.246668,0.219435
5,1001554,2018-03-22,153.0,56.7,24.2215,67,1.017,-2.730180,-0.617614,-0.995070
6,1001643,2015-08-04,176.0,63.9,20.6289,61,1.125,-0.986464,-2.195810,-2.392520
...,...,...,...,...,...,...,...,...,...,...
23522,6020460,2015-12-02,171.0,69.8,24.7307,64,1.170,-1.600300,-1.628120,-1.953780
23523,6020702,2015-09-17,160.0,74.1,28.9453,58,1.052,-2.588480,-1.163530,-1.465380
23528,6022316,2017-09-19,170.0,77.5,27.4589,73,1.237,0.424648,-0.708455,-0.463218
23531,6023119,2015-10-02,161.0,62.7,23.8912,57,1.294,1.472440,0.914185,1.294470


In [47]:
data_pm_no_nans.isna().sum()

patientId               0
date_i2                 0
height_i2               0
weight_i2             245
body_mass_index_i2    245
age_i2                  0
tbs_ls                  0
tscore_ls               0
tscore_neck             0
tscore_totalHip         0
dtype: int64

In [50]:
data_pm_no_nans.to_csv("densitometry.csv", index=False)

Upload results to project storage using ``dx upload``

In [51]:
%%bash
dx upload densitometry.csv --path /data/processed/

ID                          file-GPV6VXQJjxx5b30yPqPJxGk2
Class                       file
Project                     project-GP77K38Jjxx9XzFP2KzPQyfG
Folder                      /data/processed
Name                        densitometry.csv
State                       closing
Visibility                  visible
Types                       -
Properties                  -
Tags                        -
Outgoing links              -
Created                     Tue Feb  7 15:05:46 2023
Created by                  ollehman
 via the job                job-GPV50Y8Jjxx7Gfb87z1Fj02V
Last modified               Tue Feb  7 15:05:48 2023
Media type                  
archivalState               "live"
cloudAccount                "cloudaccount-dnanexus"
